# Visualizing Objectives



Principles:

1. Objective chains are directed and acyclic.

2. Each node represents a discrete competency cluster.

3. Each edge represents a dependency of one competency cluster on another.

4. Competency clusters should be as coherent as possible.

2. Edge chains should be independent.  I.e. one should not link to more than one previous node in a chain.  (One may link to both nodes in a forked chain.)
    
    For instance, if 303 depends on 170 and 190, but 190 depends on 170, then only 190 should be linked to 303.


##  Produce CSV from node files.

In [1]:
from glob import glob
import yaml

lessons = sorted(glob('lessons/*.yml'))

nodes = {}
for lesson in lessons:
    # whatever your local path is
    print(lesson)
    fname = f'/home/neal/urbit/curriculum/{lesson}'
    print(fname)
    with open(fname, 'r') as f:
        data = f.read()
    data = yaml.safe_load(data)
    if data is None:
        node_title = lesson[8:11]
        nodes[node_title] = {}
        nodes[node_title]['prerequisites'] = ''
        nodes[node_title]['title'] = 'Placeholder'
        nodes[node_title]['tags'] = ''
        continue
    print(data)
    node_title = lesson[8:11]
    nodes[node_title] = {}
    nodes[node_title]['prerequisites'] = data['prerequisites']
    nodes[node_title]['title'] = data['title']
    nodes[node_title]['tags'] = data['tags']

full_output = "#,Topic,Competency Cluster,,Antecedents\n"
for node in sorted(nodes.keys()):
    output = f'{node},'
    tag = repr(nodes[node]["tags"])
    if tag == '':  tag = "Placeholder"
    print(tag, tag.replace("[","\"").replace("]","\"").replace("'",""))
    output += tag.replace("[","\"").replace("]","\"").replace("'","") + ",\""
    output += f'{nodes[node]["title"]}'+'",,'
    prereq = repr(nodes[node]["prerequisites"])
    output += prereq.replace("[","\"").replace("]","\"").replace("'","")
    full_output += output + '\n'

lessons/100.yml
/home/neal/urbit/curriculum/lessons/100.yml
{'uuid': 100, 'layout': 'node', 'title': 'Administer an Urbit ship manuaully.', 'tags': ['%sysadmin'], 'prerequisites': ['0'], 'postrequisites': ['110'], 'objectives': ['Explain what an Urbit ship is.', 'Identify the `.urb/` directory.', 'Distinguish a fakeship from a liveship.', "Use the `+ls` generator to show a directory's contents.", '`|mount` and `|commit` a desk.'], 'runes': [], 'irregular': [], 'key_points': [], 'assessments': [], 'comments': '', 'content': ''}
lessons/101.yml
/home/neal/urbit/curriculum/lessons/101.yml
lessons/102.yml
/home/neal/urbit/curriculum/lessons/102.yml
{'uuid': 102, 'layout': 'node', 'title': 'Description of address space structure.', 'tags': ['%azimuth'], 'prerequisites': ['0'], 'postrequisites': ['112', '302'], 'objectives': ['Understand the role of the public-key infrastructure in Urbit.', 'Describe the high-level architecture of the Urbit ID address space and distinguish types of points.',

In [2]:
print(full_output)
with open('curr-raw.csv', 'w') as outfile:
    outfile.write(full_output)

#,Topic,Competency Cluster,,Antecedents
100,"%sysadmin","Administer an Urbit ship manuaully.",,"0"
101,,"Placeholder",,
102,"%azimuth","Description of address space structure.",,"0"
103,"%hoon","Aural Hoon",,"0"
110,"%hoon","Syntax, nouns, auras",,"100, 103"
112,"%azimuth","Exploration of address space structure.",,"102, 110"
113,"%hoon","Naked generator design, syncing, etc., control structures",,"110"
115,"%hoon","Irregular forms, sugar syntax",,"113"
120,"%hoon","Basic dry gates",,"115"
125,"%hoon","Molds, typechecking",,"120"
130,"%hoon","Traps, recursion",,"125"
133,"%hoon","Basic cores, arms",,"130"
135,"%hoon","Addressing, trees",,"133"
140,"%stdlib","Lists, trees",,"135"
145,"%hoon","Libraries",,"140"
150,"%hoon","Doors",,"145"
153,"%hoon","The beak",,"145"
155,"%hoon","Revisiting Cores",,"150"
156,"%hoon","Typechecking",,"133"
160,"%stdlib","Basic text transformations",,"125"
163,"%hoon","Producing textually rich %say generators.",,"145"
165,"%hoon","Subject-oriented programmi

##  Load and clean CSV of nodes

In [3]:
# Load and clean CSV of nodes and descriptions.
from csv import DictReader

nodes = {}
edges = {}
with open('curr-raw.csv', 'r') as rawFile:
    rawData = DictReader(rawFile)
    
    # Build nodes.
    for line in rawData:
        if line["Topic"] in ['', 'Aggregate'] or line["Competency Cluster"] in '': continue
        nodes[line['#']] = f'{line["Topic"]}: {line["Competency Cluster"]}'
        
        if line['Antecedents'].strip() == '': continue
        edges[line['#']] = []
        ants = line['Antecedents'].split(',')
        for ante in ants:
            if line["Antecedents"].strip() == '0': continue
            edges[line['#']].append(ante.strip())
            #print(line['#'], edges[line['#']])

In [4]:
nodes

{'100': '%sysadmin: Administer an Urbit ship manuaully.',
 '102': '%azimuth: Description of address space structure.',
 '103': '%hoon: Aural Hoon',
 '110': '%hoon: Syntax, nouns, auras',
 '112': '%azimuth: Exploration of address space structure.',
 '113': '%hoon: Naked generator design, syncing, etc., control structures',
 '115': '%hoon: Irregular forms, sugar syntax',
 '120': '%hoon: Basic dry gates',
 '125': '%hoon: Molds, typechecking',
 '130': '%hoon: Traps, recursion',
 '133': '%hoon: Basic cores, arms',
 '135': '%hoon: Addressing, trees',
 '140': '%stdlib: Lists, trees',
 '145': '%hoon: Libraries',
 '150': '%hoon: Doors',
 '153': '%hoon: The beak',
 '155': '%hoon: Revisiting Cores',
 '156': '%hoon: Typechecking',
 '160': '%stdlib: Basic text transformations',
 '163': '%hoon: Producing textually rich %say generators.',
 '165': '%hoon: Subject-oriented programming',
 '170': '%testing: Basic debugging techniques',
 '175': '%hoon: Building code',
 '180': '%hoon: State: tisket etc.',


##  Convert to graph representation

In [5]:
import pygraphviz as pgv

G = pgv.AGraph(edges,
               strict=True,
               directed=True,
               rankdir="RL",
               ranksep="0.25",
               ordering="in")

G.layout('dot')
G.draw('curr.png')
G.draw('curr.svg')

##  Manually build palette for Hoon School topics

In [6]:
data = '''100
110, 113, 103
115, 120, 125, 175
130, 133, 135
140, 145, 160
150, 153, 155, 165
156, 183
184, 233
163, 180
170, 190, 217'''

colors = {
    0:'#9e0142',
    1:'#d53e4f',
    2:'#f46d43',
    3:'#fdae61',
    4:'#fee08b',
    5:'#e6f598',
    6:'#abdda4',
    7:'#66c2a5',
    8:'#3288bd',
    9:'#5e4fa2',
}
#http://colrd.com/palette/18894/ "Spectral 10" by 
nodes = {}
for idx,row in enumerate(data.split('\n')):
    for val in row.split(','):
        nodes[val.strip()] = colors[idx]

##  Mark SVG nodes by color.

In [7]:
from bs4 import BeautifulSoup
import re

with open('curr.svg') as f:
    svg = f.read()

soup = BeautifulSoup(svg, 'lxml')

svg_tag = soup.find('svg')
svg_grf = svg_tag.find('g')
paths = svg_grf.find_all('g', {'class': 'node'})

for path in paths:
    try:
        title = path.findChild('title')
    except:
        continue
    if title.contents[0] in nodes:
        ellipse = path.findChild('ellipse')
        color = nodes[title.contents[0]]
        ellipse['fill'] = f'{color}'
        if title.contents[0] == '217':
            print('here')
            #style="fill: url(#circles-1) #fff;"
            ellipse['fill'] = f'url(#diagonalHatch) {color}';
            pass

with open('curr-marked.svg', 'w') as f:
    svg = '''<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">'''
    color = colors[9]
    svg_str = str(svg_tag)
    idx = svg_str.find('<g')
    svg_hdr = svg_str[:idx]
    svg_defs = f'''<defs><pattern id="diagonalHatch" width="4" height="8" patternTransform="rotate(-45 2 2)" patternUnits="userSpaceOnUse"><path d="M -1,2 l 6,0" stroke="{color}" stroke-width="4"/></pattern></defs>'''
    svg_str = svg+'\n'+svg_hdr+svg_defs+'\n'+svg_str[idx:]
    f.write(svg_str)
    
from cairosvg import svg2png
svg2png(bytestring=(svg+str(svg_tag)), dpi=100, write_to='curr-marked.png')

here


##  Build objectives list.

In [8]:
data = '''100
110, 113, 103
115, 120, 125, 175
130, 133, 135
140, 145, 160
150, 155, 165
156, 183
184, 233
163, 180
170, 190, 217'''

lessons = {}
for idx,row in enumerate(data.split('\n')):
    lessons[idx] = []
    for val in row.split(','):
        lessons[idx].append(val.strip())

import yaml
for lesson in lessons:
    print(f'After Lesson {lesson}, you should be able to:\n')
    for node in lessons[lesson]:
        # whatever your local path is
        fname = f'/home/neal/urbit/curriculum/lessons/{node}.yml'
        with open(fname, 'r') as f:
            data = f.read()
        data = yaml.safe_load(data)
        for obj in data['objectives']:
            print('-', obj)
    
    print('\nYou will know the runes:\n')
    for node in lessons[lesson]:
        # whatever your local path is
        fname = f'/home/neal/urbit/curriculum/lessons/{node}.yml'
        with open(fname, 'r') as f:
            data = f.read()
        data = yaml.safe_load(data)
            
        for rune in data['runes']:
            print(f'- `{rune}`')
    print('\n---\n')

After Lesson 0, you should be able to:

- Explain what an Urbit ship is.
- Identify the `.urb/` directory.
- Distinguish a fakeship from a liveship.
- Use the `+ls` generator to show a directory's contents.
- `|mount` and `|commit` a desk.

You will know the runes:


---

After Lesson 1, you should be able to:

- Distinguish nouns, cells, and atoms.
- Apply auras to transform an atom.
- Identify common Hoon molds, such as cells, lists, and tapes.
- Annotate Hoon code with comments.
- Pin a face to the subject.
- Make a decision at a branch point.
- Distinguish loobean from boolean operations.
- Slam a gate (call a function).
- Produce a generator to convert a value between auras.
- Pronounce ASCII characters per standard Hoon developer practice.

You will know the runes:

- `::`
- `%-`
- `=/`
- `?:`
- `^-`
- `~&`
- ``

---

After Lesson 2, you should be able to:

- Identify current known irregular syntax.
- Convert between regular and irregular forms of runes to date.
- Employ a gate t

In [9]:
lessons

{0: ['100'],
 1: ['110', '113', '103'],
 2: ['115', '120', '125', '175'],
 3: ['130', '133', '135'],
 4: ['140', '145', '160'],
 5: ['150', '155', '165'],
 6: ['156', '183'],
 7: ['184', '233'],
 8: ['163', '180'],
 9: ['170', '190', '217']}

In [10]:
topics = '''Aggregate
Ames
API
Arvo
%ask generators
Azimuth
Behn
Clay
CLI
Dill
Distribution
Eyre
Gall
Gall agents
Gall (Spider)
Hoon
Iris
Jael
Khan
Nock
Runtime
Sail
Stdlib
Sysadmin
Testing'''.split('\n')

# Load and clean CSV of nodes and descriptions.
from csv import DictReader

nodes = {}
edges = {}
topicd = {}
with open('curr-raw.csv', 'r') as rawFile:
    rawData = DictReader(rawFile)
    
    # Build nodes.
    for line in rawData:
        if line["Topic"] in ['', 'Aggregate'] or line["Competency Cluster"] in '': continue
        nodes[line['#']] = f'{line["Topic"]}: {line["Competency Cluster"]}'
        
        if line['Antecedents'].strip() == '': continue
        edges[line['#']] = []
        ants = line['Antecedents'].split(',')
        for ante in ants:
            if line["Antecedents"].strip() == '0': continue
            edges[line['#']].append(ante.strip())
            #print(line['#'], edges[line['#']])
        
        if line["Topic"] in topics:
            topic = line['Topic']
            if topic == 'Distribution':  topic = 'Clay'
            if 'Gall' in topic:  topic = 'Gall'
            if 'API' in topic:  topic = 'Stdlib'
            if 'CLI' in topic:  topic = 'Khan'
            if '%ask' in topic:  topic = 'Hoon'
            if 'Sail' in topic:  topic = 'Hoon'
            topicd[line["#"]] = topic

##  Mark SVG nodes by topic

In [11]:
# Hoon/Stdlib/API.CLI are solid color
# Arvo+vanes are striped

colors = [
    '#9e0142',
    '#d53e4f',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#e6f598',
    '#abdda4',
    '#66c2a5',
    '#3288bd',
    '#5e4fa2',
]

from numpy.random import choice
colors_ = choice(colors, size=(10,), replace=False)
colors = {
    'Ames':colors_[0],
    'Behn':colors_[1],
    'Clay':colors_[2],
    'Dill':colors_[3],
    'Eyre':colors_[4],
    'Gall':colors_[5],
    'Iris':colors_[6],
    'Jael':colors_[7],
    'Khan':colors_[8],
    'Arvo':colors_[9],
    'Hoon':colors_[5],
    'Azimuth':colors_[7],
    'Nock':'#ffffff',
    'Runtime':colors_[9],
    'Stdlib':colors_[5],
    'Sysadmin':colors_[3],
    'Testing':colors_[8],
}
hatched = ['Ames', 'Behn', 'Clay', 'Dill', 'Eyre', 'Gall', 'Iris', 'Jael', 'Khan', 'Arvo']

svg_defs = ''
for h in hatched:
    svg_defs += f'''<defs><pattern id="diagonalHatch{h}" width="4" height="8" patternTransform="rotate(-45 2 2)" patternUnits="userSpaceOnUse"><path d="M -1,2 l 6,0" stroke="{colors[h]}" stroke-width="4"/></pattern></defs>'''


from bs4 import BeautifulSoup
import re

with open('curr.svg') as f:
    svg = f.read()

soup = BeautifulSoup(svg, 'lxml')

svg_tag = soup.find('svg')
svg_grf = svg_tag.find('g')
paths = svg_grf.find_all('g', {'class': 'node'})

for path in paths:
    try:
        title = path.findChild('title')
    except:
        continue
    if title.contents[0] in topicd.keys():
        ellipse = path.findChild('ellipse')
        color = colors[topicd[title.contents[0]]]
        ellipse['fill'] = f'{color}'
        if topicd[title.contents[0]] in hatched:
            ellipse['fill'] = f'url(#diagonalHatch{topicd[title.contents[0]]}) {color}';

with open('curr-topics.svg', 'w') as f:
    svg = '''<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">'''
    color = colors['Testing']
    svg_str = str(svg_tag)
    idx = svg_str.find('<g')
    svg_hdr = svg_str[:idx]
    svg_str = svg+'\n'+svg_hdr+svg_defs+'\n'+svg_str[idx:]
    f.write(svg_str)
    
from cairosvg import svg2png
svg2png(bytestring=(svg+str(svg_tag)), dpi=100, write_to='curr-topics.png')

print(colors)

{'Ames': '#fee08b', 'Behn': '#fdae61', 'Clay': '#66c2a5', 'Dill': '#3288bd', 'Eyre': '#5e4fa2', 'Gall': '#d53e4f', 'Iris': '#f46d43', 'Jael': '#abdda4', 'Khan': '#9e0142', 'Arvo': '#e6f598', 'Hoon': '#d53e4f', 'Azimuth': '#abdda4', 'Nock': '#ffffff', 'Runtime': '#e6f598', 'Stdlib': '#d53e4f', 'Sysadmin': '#3288bd', 'Testing': '#9e0142'}


In [12]:
svg_str = f'''
<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">

'''

with open('legend.svg', 'w') as f:
    svg = ''''''
    

# Manually Build App School topics.

In [13]:
data = '''100,
282,284
382
288,299
314,330
375,482,external
200,205,210,215,220,225,230,235,240,245
430
190,303,217,390
'''

colors = {
    0:'#9e0142',
    1:'#d53e4f',
    2:'#f46d43',
    3:'#fdae61',
    4:'#fee08b',
    5:'#e6f598',
    6:'#abdda4',
    7:'#66c2a5',
    8:'#3288bd',
    9:'#5e4fa2',
}
#http://colrd.com/palette/18894/ "Spectral 10" by 
nodes = {}
for idx,row in enumerate(data.split('\n')):
    for val in row.split(','):
        nodes[val.strip()] = colors[idx]

# Mark SVG nodes by color.

In [14]:
from bs4 import BeautifulSoup
import re

with open('curr.svg') as f:
    svg = f.read()

soup = BeautifulSoup(svg, 'lxml')

svg_tag = soup.find('svg')
svg_grf = svg_tag.find('g')
paths = svg_grf.find_all('g', {'class': 'node'})

for path in paths:
    try:
        title = path.findChild('title')
    except:
        continue
    if title.contents[0] in nodes:
        ellipse = path.findChild('ellipse')
        color = nodes[title.contents[0]]
        ellipse['fill'] = f'{color}'
        if title.contents[0] == '217':
            print('here')
            #style="fill: url(#circles-1) #fff;"
            ellipse['fill'] = f'url(#diagonalHatch) {color}';
            pass

with open('curr-asl.svg', 'w') as f:
    svg = '''<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">'''
    color = colors[9]
    svg_str = str(svg_tag)
    idx = svg_str.find('<g')
    svg_hdr = svg_str[:idx]
    svg_defs = f'''<defs><pattern id="diagonalHatch" width="4" height="8" patternTransform="rotate(-45 2 2)" patternUnits="userSpaceOnUse"><path d="M -1,2 l 6,0" stroke="{color}" stroke-width="4"/></pattern></defs>'''
    svg_str = svg+'\n'+svg_hdr+svg_defs+'\n'+svg_str[idx:]
    f.write(svg_str)
    
from cairosvg import svg2png
svg2png(bytestring=(svg+str(svg_tag)), dpi=100, write_to='curr-asl.png')

here
